# 赛题介绍

百度网盘AI大赛——图像处理挑战赛是 百度网盘开放平台 面向AI开发者和爱好者发起的计算机视觉领域挑战赛。AI时代已到来，百度网盘与百度飞桨AI Studio强强联合，旨在基于个人云存储的生态能力开放，通过比赛机制，鼓励选手结合当下前沿的计算机视觉技术与图像处理技术，完成模型设计搭建与训练优化，产出基于飞桨框架的开源模型方案，为中国开源生态建设贡献一份力量。

本次图像处理挑战赛以线上比赛的形式进行，参赛选手需要在规定时间内，完成模型设计搭建与训练优化，并提交基于评测数据集产出的结果文件，榜单排名靠前且通过代码复查的队伍可获得高额奖金。

百度网盘开放平台 致力于为全球AI开发者和爱好者提供专业、高效的AI学习与开发环境，挖掘培养AI人才，助力技术产业生态发展。我们向所有胸怀梦想的开发者发出邀请，希望大家能与百度一起，用技术让复杂的世界变得更简单。

# 数据集简介

选手需要建立模型，对比赛给定的带有摩尔纹的图片进行处理，消除屏摄产生的摩尔纹噪声，还原图片原本的样子，并提交模型输出的结果图片。希望各位参赛选手结合当下前沿的计算机视觉技术与图像处理技术，在设计搭建模型的基础上，提升模型的训练性能、精度效果和泛化能力。在保证效果精准的同时，可以进一步考虑模型在实际应用中的性能表现，如更轻量、更高效等。

在本次比赛最新发布的数据集中，所有的图像数据均由真实场景采集得到，再通过技术手段进行相应处理，生成可用的脱敏数据集。该任务为image-to-image的形式，因此源数据和GT数据均以图片的形式来提供。各位选手可基于本次比赛最新发布的训练数据快速融入比赛，为达到更好的算法效果，本次比赛不限制大家使用额外的训练数据来优化模型。测试数据集的GT不做公开，请各位选手基于本次比赛最新发布的测试数据集提交对应的结果文件。

数据集构成
```
|- root  
    |- images
    |- gts
```
本次比赛最新发布的数据集共包含训练集、A榜测试集、B榜测试集三个部分，其中训练集共1000个样本，A榜测试集共200个样本，B榜测试集共200个样本；
images 为带摩尔纹的源图像数据，gts 为无摩尔纹的真值数据（仅有训练集数据提供gts ，A榜测试集、B榜测试集数据均不提供gts）；
images 与 gts 中的图片根据图片名称一一对应。




# Baseline模型WDNet介绍

## 整体结构
WDNet是ECCV 2020提出一种去除摩尔纹的模型。该模型是一种基于小波与双分支的神经网络，结构如下：

![](https://ai-studio-static-online.cdn.bcebos.com/1352cb0a68d14622a2b2e5d2ec3f3edb82deea6547b54eaf8aa5bb0a9e22ed24)

首先RGB图片需要通过WaveletTransform模块进行转换，得到一个48通道的数据，通过WDNet网络同样得到一个通道数与尺寸不变的特征图。最后在一次通过WaveletTransform使用转置卷积将图片还原得到最终预测结果。

这里WaveletTransform的权重是固定不变不需要训练的。

## DenseNet

![](https://ai-studio-static-online.cdn.bcebos.com/9ef3defdba2f4b15ba6bdde87cda1a724530b8cb3ade4593ad9637593f58b39c)

DenseNet中使用旁路连接和特征复用的方式缓解了梯度消失的问题，同时减少了网络参数。DenseNet已经被用于去雾和超分辨率网络。

如上图所示，该模型中的dense分支新增了一个方向感知模块（DPM），用于找到摩尔纹的方向。DPM的输出和每一个dense的输出相乘，然后乘以一个因子β然后与输入相加。该设计可以有效的定位摩尔纹的位置。


## Dilation
![](https://ai-studio-static-online.cdn.bcebos.com/8545d61038ad41658c5610ed14b9bcfa6ae9c1947cb9469c80fc2ccfb62b9b55)

下采样和池化可以增大感受野，但同时也丢失了一些细节。空洞卷积可以解决这个问题。在每一个dilation分支里，都有两层，有一个3x3的空洞卷积和3x3的普通卷积组成。

# 训练

## 准备数据集



In [1]:
%cd /home/aistudio/data/
!unzip /home/aistudio/data/data120844/moire_train_dataset.zip
!unzip /home/aistudio/data/data120844/moire_testA_dataset.zip

/home/aistudio/data
Archive:  /home/aistudio/data/data120844/moire_train_dataset.zip
   creating: moire_train_dataset/
  inflating: __MACOSX/._moire_train_dataset  
  inflating: moire_train_dataset/.DS_Store  
  inflating: __MACOSX/moire_train_dataset/._.DS_Store  
   creating: moire_train_dataset/images/
  inflating: __MACOSX/moire_train_dataset/._images  
   creating: moire_train_dataset/gts/
  inflating: __MACOSX/moire_train_dataset/._gts  
  inflating: moire_train_dataset/images/moire_train_00676.jpg  
  inflating: __MACOSX/moire_train_dataset/images/._moire_train_00676.jpg  
  inflating: moire_train_dataset/images/moire_train_00110.jpg  
  inflating: __MACOSX/moire_train_dataset/images/._moire_train_00110.jpg  
  inflating: moire_train_dataset/images/moire_train_00104.jpg  
  inflating: __MACOSX/moire_train_dataset/images/._moire_train_00104.jpg  
  inflating: moire_train_dataset/images/moire_train_00662.jpg  
  inflating: __MACOSX/moire_train_dataset/images/._moire_train_00662.jp

解压数据集之后，数据集目录如下
```
moire_train_dataset/
├── gts
└── images
```

## 启动训练命令

In [2]:
%cd /home/aistudio/demoire-baseline/
!python train.py --dataset_root ../data/moire_train_dataset/ --max_epochs 200 --batch_size 4 --sample_interval 100

/home/aistudio/demoire-baseline
W0105 18:06:10.064575   241 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0105 18:06:10.068404   241 device_context.cc:465] device: 0, cuDNN Version: 7.6.
Loading pretrained model from ./vgg.pdparams
There are 38/38 variables loaded into VGG.
[Epoch 200/200] [Batch 240/250] [G loss: 10.224015, pixel: 0.012445] ETA: -1 day, 23:56:34.751816

# 预测

## 启动预测脚本

In [5]:
%cd /home/aistudio/demoire-baseline/
!python predict.py --dataset_root ../data/moire_testA_dataset/ --pretrained train_result/model/epoch_200/model.pdparams 

/home/aistudio/demoire-baseline
W0106 08:10:19.632091 59131 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0106 08:10:19.635650 59131 device_context.cc:465] device: 0, cuDNN Version: 7.6.
Loading pretrained model from train_result/model/epoch_200/model.pdparams
There are 176/176 variables loaded into WDNet.


In [3]:
%cd /home/aistudio/demoire-baseline/
!python predict_samesize.py --dataset_root ../data/moire_testA_dataset/ --pretrained train_result/model/epoch_200/model.pdparams 

/home/aistudio/demoire-baseline
W0106 07:48:03.332212 57318 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0106 07:48:03.335692 57318 device_context.cc:465] device: 0, cuDNN Version: 7.6.
Loading pretrained model from train_result/model/epoch_200/model.pdparams
There are 176/176 variables loaded into WDNet.


运行结束后，结果文件保存在output/pre目录下。进入pre目录创建readme.txt文件。输入以下内容：

```
训练框架：PaddlePaddle
代码运行环境：V100
是否使用GPU：是
单张图片耗时/s：0.5
模型大小：15MB
其他说明：该算法参考WDNET网络。
```

然后压缩所有文件。

In [6]:
%cd /home/aistudio/demoire-baseline/output/pre/
!zip result.zip *.jpg *.txt

/home/aistudio/demoire-baseline/output/pre
	zip warning: name not matched: *.txt
updating: moire_testA_00000.jpg (deflated 1%)
updating: moire_testA_00001.jpg (deflated 1%)
updating: moire_testA_00002.jpg (deflated 0%)
updating: moire_testA_00003.jpg (deflated 1%)
updating: moire_testA_00004.jpg (deflated 0%)
updating: moire_testA_00005.jpg (deflated 0%)
updating: moire_testA_00006.jpg (deflated 0%)
updating: moire_testA_00007.jpg (deflated 0%)
updating: moire_testA_00008.jpg (deflated 0%)
updating: moire_testA_00009.jpg (deflated 0%)
updating: moire_testA_00010.jpg (deflated 0%)
updating: moire_testA_00011.jpg (deflated 0%)
updating: moire_testA_00012.jpg (deflated 0%)
updating: moire_testA_00013.jpg (deflated 1%)
updating: moire_testA_00014.jpg (deflated 0%)
updating: moire_testA_00015.jpg (deflated 0%)
updating: moire_testA_00016.jpg (deflated 0%)
updating: moire_testA_00017.jpg (deflated 0%)
updating: moire_testA_00018.jpg (deflated 0%)
updating: moire_testA_00019.jpg (deflated 0%)

最后下载result.zip文件，提交即可。最终得分57.07。